In [ ]:
cd 'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Biofilm growth code\GSWL code\Biochemical-databases'

Export a condensed form of compound names from the ModelSEED database. 

In [23]:
from datetime import date
import json
import re


modelseed = json.load(open('compounds.json'))
modelseed_compounds = {}
all_names = []
for compound2 in modelseed:
    if compound2['is_obsolete'] == 0:
        try:
            for entry in compound2['aliases']:
                if re.search('Name:', entry):
                    alternative_names = re.sub('Name: ', '', entry).split('; ')
                    alternative_names.append(compound2['abbreviation'])
                    main_name = alternative_names[0].lower()
                if re.search('KEGG:', entry):
                    kegg_id = re.sub('KEGG: ', '', entry).split('; ')

        except:
            main_name = compound2['name'].lower()
            alternative_names = compound2['abbreviation']
            
        modelseed_compounds[main_name] = {'ModelSEED ID': compound2['id'],
                                          'KEGG ID': kegg_id,
                                          'alternative names': alternative_names}

# export the dictionary of modelseed compounds with the respective information
with open('{}_modelseed compounds & alternative names_01.json'.format(date.today()), 'w') as output:
    json.dump(modelseed_compounds, output, indent = 4)

The lists of NIST compounds that are described and undescribed in the ModelsEED database are exported. These compounds are paired with their respective ModelSEED IDs in later code.  

In [81]:
# import the requisite libraries
from ftfy import fix_encoding
from datetime import date
import numpy
import json
import pandas
import re

modelseed = json.load(open('2021-06-27_modelseed compounds & alternative names_01.json'))
compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

empty_cells = ['null', ' null', numpy.nan, 'None', None, ""]

# parse the modelseed database and the list of thermodynamically described compounds
'''nist_compounds = []
for compound in compounds:
    nist_compounds.append(compound)
nist_compounds_string = ', '.join(nist_compounds)
print(nist_compounds_string)'''


    
#print(modelseed_abbreviations)
all_names = []
for compound in modelseed:
    if type(modelseed[compound]['alternative names']) is list:
        all_names.append([compound.lower()] + modelseed[compound]['alternative names'])
    elif type(modelseed[compound]['alternative names']) is str:
        all_names.append([compound.lower()] + list(modelseed[compound]['alternative names']))
    else:
        print('ERROR: The database entry for {} is an unpredicted structure.')
        
total_modelseed_names = []
for name_list in all_names:
    total_modelseed_names += name_list 

total_modelseed_names_string = ',\t'.join(list(set(total_modelseed_names)))   
#    if all(name not in described_compounds.keys() for name in all_names):
'''with open('2021-06-27_total_modelseed_names_string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''

modelseed_abbreviations = modelseed.keys()
modelseed_abbreviations_string = ',\t'.join(modelseed_abbreviations)

'''with open('2021-06-27_modelseed abbreviation string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''


described_compounds = {}
undescribed_compounds = []
unknown_compounds = []
repeated_compounds = set()
for compound in compounds:    
    compound = re.sub('(\(\+\))', '\(\+\)', compound)
    compound = compound.lower()
    if compound not in (described_compounds.keys() or undescribed_compounds or unknown_compounds):
        '''try:'''
        if re.search('((^|,\t){},\t)'.format(compound), total_modelseed_names_string, re.IGNORECASE):
            if re.search('((^|,\t){},\t)'.format(compound), modelseed_abbreviations_string, re.IGNORECASE):
                compound = fix_encoding(compound)
                print('identified abbreviation: ', compound)
                #re.sub(',\s(\w)\w+', '\U\1', ', {}, '.format(compound))
                compound = re.sub('(\\\)', '', compound)
                described_compounds[compound] = modelseed[compound]
            else:
                for name_list in all_names:
                    name_list_string = ',\t'.join(name_list)
                    if re.search('((^|,\t){},\t)'.format(compound), name_list_string, re.IGNORECASE):
                        root_compound = name_list[0]
                        root_compound = fix_encoding(root_compound)
                        print('identified alias: ', compound)
                        #print(name_string)
                        described_compounds[compound] = modelseed[root_compound]
    
                if compound not in (described_compounds.keys() or unknown_compounds):
                    unknown_compounds.append(compound)
                    print('ERROR 1: {} is structurally predicted.'.format(compound))

        else:
            if compound not in undescribed_compounds:
                compound = fix_encoding(compound)
                undescribed_compounds.append(compound)
                print('--> {} is undescribed by ModelSEED'.format(compound))
            else:
                if compound not in unknown_compounds:
                    unknown_compounds.append(compound)
                    print('ERROR 2: {} is structurally predicted.'.format(compound))
                    
    else:
        repeated_compounds.add(compound)
                
        '''except:
            print('ERROR: The {} compound is incompatible with the regex search function .'.format(compound))'''
        
#total_undescribed_compounds = undescribed_compounds + unknown_compounds

print('described: ', len(described_compounds.keys()))
print('unknown compounds: ', len(unknown_compounds))
print('undescribed_compounds: ', len(undescribed_compounds))
print('repeated compounds: ', len(repeated_compounds))

# export the acquired set of described compounds
with open('{}_APF_described compounds in ModelSEED_01.json'.format(date.today()), 'w') as output:
    json.dump(described_compounds, output, indent = 5)
    
# export the set of undescribed compounds
with open('{}_APF_undescribed compounds in ModelSEED_01.csv'.format(date.today()), 'w', encoding="utf-8") as output:
    undescribed_dataframe = pandas.DataFrame(undescribed_compounds, columns = ['undescribed compounds'])
    undescribed_dataframe.fillna('', inplace = True)
    undescribed_dataframe.to_csv(output)

identified alias:  l-aspartate
identified abbreviation:  fumarate
identified abbreviation:  ammonia
identified alias:  h2o
--> (s)-malate is undescribed by ModelSEED
identified alias:  2-phospho-d-glycerate
identified alias:  phosphoenolpyruvate
identified abbreviation:  d-fructose 1,6-bisphosphate
identified alias:  glycerone phosphate
identified alias:  3-phospho-d-glycerate
identified alias:  phospho-l-arginine
identified alias:  l-arginine
identified alias:  orthophosphate
identified alias:  phosphocreatine
identified abbreviation:  creatine
--> (r)-3-phosphoglycerate is undescribed by ModelSEED
--> (r)-glycerate is undescribed by ModelSEED
identified alias:  pyruvate
identified abbreviation:  d-fructose 1-phosphate
identified alias:  d-fructose 6-phosphate
identified alias:  d-fructose 6-phosphate
identified abbreviation:  d-fructose
identified alias:  d-galactose 6-phosphate
identified abbreviation:  d-galactose
identified abbreviation:  d-glucose 6-phosphate
identified abbreviat

identified alias:  damp
identified alias:  datp
--> adp-n1-oxide is undescribed by ModelSEED
--> atp-n1-oxide is undescribed by ModelSEED
identified abbreviation:  adenosine 5'-tetraphosphate
identified alias:  citrate
identified alias:  cis-aconitate
identified alias:  isocitrate
identified alias:  isocitrate
identified abbreviation:  acetate
identified abbreviation:  acetyl phosphate
identified alias:  3-phospho-d-glyceroyl phosphate
identified alias:  (aminomethyl)phosphonate
--> (3-phospho-d-glycerol-aminomethyl)phosphonate is undescribed by ModelSEED
--> (chloroethyl)phosphonate is undescribed by ModelSEED
--> (3-phospho-d-glycerol-chloroethyl)phosphonate is undescribed by ModelSEED
--> (chloromethyl)phosphonate is undescribed by ModelSEED
--> (3-phospho-d-glycerol-chloromethyl)phosphonate is undescribed by ModelSEED
identified alias:  (ethyl)phosphonate
--> (3-phospho-d-glycerol-ethyl)phosphonate is undescribed by ModelSEED
--> (methoxy)phosphonate is undescribed by ModelSEED
-->

--> (n-1) h2o is undescribed by ModelSEED
identified alias:  l-lysine
--> nα-benzoyl-l-argininamide is undescribed by ModelSEED
--> nα-benzoyl-l-arginine is undescribed by ModelSEED
identified abbreviation:  d-mannitol 1-phosphate
identified alias:  hydroxypyruvate
identified alias:  glycine
identified alias:  l-serine
identified alias:  l-threonine
identified alias:  5,10-methylenetetrahydrofolate
identified alias:  tetrahydrofolate
--> ch2-h4pteglu4 is undescribed by ModelSEED
identified abbreviation:  h4pteglu4
identified abbreviation:  l-histidinol phosphate
--> 3-(imidazol-4-yl)-2-oxopropyl phosphate is undescribed by ModelSEED
--> γ-/glutamohydroxamic acid is undescribed by ModelSEED
identified abbreviation:  l-ribulose 5-phosphate
identified abbreviation:  d-xylulose 5-phosphate
identified alias:  l-rhamnose
identified alias:  l-rhamnulose
identified abbreviation:  3'-dephospho-coa
identified alias:  all-trans-retinal
identified abbreviation:  11-cis-retinal
--> d-ribulose 1,5-b

identified abbreviation:  d-serine
--> h2o(1) is undescribed by ModelSEED
identified abbreviation:  2-methylserine
--> (r)-methylmalonyl-coa is undescribed by ModelSEED
identified alias:  thymidine
identified abbreviation:  2'-deoxyadenosine
identified alias:  thymine
identified alias:  l-carnitine
--> l-acetylcarnitine is undescribed by ModelSEED
identified alias:  butyryl-coa
--> l-butyrylcarnitine is undescribed by ModelSEED
identified alias:  propionyl-coa
--> l-propionylcarnitine is undescribed by ModelSEED
identified alias:  5,6,7,8-tetrahydrofolate
identified abbreviation:  7,8-dihydrofolate
identified alias:  folate
identified alias:  folate
identified alias:  4-hydroxy-2-oxoglutarate
identified abbreviation:  5-oxo-d-proline
--> γ-fluoroglutamate is undescribed by ModelSEED
--> cis-d-3-fluoro-2-pyrrolidone-5-carboxylate is undescribed by ModelSEED
--> trans-d-3-fluoro-2-pyrrolidone-5-carboxylate is undescribed by ModelSEED
--> 5-oxo-d-gluconate is undescribed by ModelSEED
iden

--> α,α-trehalose is undescribed by ModelSEED
identified abbreviation:  h2o2
--> orthophospate is undescribed by ModelSEED
--> (r,s)-malyl-coa is undescribed by ModelSEED
identified alias:  dump
identified alias:  dihydrofolate
identified alias:  dtmp
identified abbreviation:  d-ornithine
identified alias:  d-threo-2,4-diaminopentanoate
identified abbreviation:  d-arabino-3-hexulose 6-phosphate
identified alias:  4,5-dihydro-4-oxo-5-imidazolepropanoate
--> n-acetyl-l-phenylalanine-l-dibromidetyrosine ethyl ester is undescribed by ModelSEED
--> l-dibromotyrosine ethyl ester is undescribed by ModelSEED
--> n-acetyl-l-phenylalanyl-l-phenylalanylglycine is undescribed by ModelSEED
--> l-phenylalanylglycine is undescribed by ModelSEED
--> n-acetyl-l-phenylalanyl-l-phenylalanylglycine methyl ester is undescribed by ModelSEED
--> l-phenylalanylglycine methyl ester is undescribed by ModelSEED
identified alias:  o-phosphorylethanolamine
identified alias:  cdpethanolamine
identified alias:  coni

--> (r)-2-nonyl butyrate is undescribed by ModelSEED
--> (r)-2-octanol is undescribed by ModelSEED
--> (r)-2-octyl butyrate is undescribed by ModelSEED
--> (r)-2-undecanol is undescribed by ModelSEED
--> (r)-2-undecyl butyrate is undescribed by ModelSEED
--> 3,4,5-trihydroxybenzoic acid propyl ester is undescribed by ModelSEED
identified abbreviation:  3,4,5-trihydroxybenzoate
identified alias:  propan-1-ol
identified abbreviation:  prephenate
identified alias:  3-aminopropionaldehyde
identified alias:  1,3-diaminopropane
identified alias:  o-phosphocholine
identified abbreviation:  4-hydroxybenzoate
identified abbreviation:  phenylpyruvate
identified alias:  4-hydroxyphenylpyruvate
identified alias:  oxonic acid
--> dihydroxonic acid is undescribed by ModelSEED
described:  521
unknown compounds:  0
undescribed_compounds:  382
repeated compounds:  0


The sets of determined and undetermined compounds are contrasted. 

In [85]:
from ftfy import fix_encoding
import pandas
import json
import io

compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

compounds_list = []
for compound in compounds.keys():
    compound = fix_encoding(compound)
    #print(compound)
    compounds_list.append(compound.lower())
                          
undescribed_compounds = pandas.read_csv('2021-06-27_APF_undescribed compounds in ModelSEED_01.csv')
#display(undescribed_compounds.head())                   
undescribed_compounds_set = set()
duplicated_compounds = []
#print('\n\n')
for compound in undescribed_compounds['undescribed compounds']:
    compound = fix_encoding(compound)
    compound = compound.strip()
    undescribed_compounds_set.add(compound)

described_compounds = json.load(open('2021-06-27_APF_described compounds in ModelSEED_01.json'))

print('Described compounds: ', len(described_compounds.keys()))
print('Undescribed compounds: ', len(undescribed_compounds['undescribed compounds']))
print('Total compounds: ', len(set(compounds_list)))

#print('\n')
modeled_compounds = undescribed_compounds_set.union(set(described_compounds.keys())) 
modeled_compounds_set = set()
for compound in modeled_compounds:
    compound = fix_encoding(compound)
    compound = compound.strip()
    modeled_compounds_set.add(compound)
    
    
print('modeled compounds: ', len(modeled_compounds_set))
missing_compounds = set(compounds_list) - modeled_compounds_set
print('Missing compounds: ', len(set(missing_compounds)))
for compound in missing_compounds:
    print(compound)


    
'''undescribed_compounds = list(set(compounds_list) - set(described_compounds.keys()))
print('Undescribed compounds: {}\n'.format(len(undescribed_compounds)), '='*len('Undescribed compounds:'))
for compound in undescribed_compounds:
    print(compound)'''
    
    
'''with io.open('2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv', 'w', encoding="utf-8") as output:
    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)
    undescribed_compund_csv.to_csv(output)'''

Described compounds:  521
Undescribed compounds:  382
Total compounds:  903
modeled compounds:  903
Missing compounds:  0


'with io.open(\'2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv\', \'w\', encoding="utf-8") as output:\n    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)\n    undescribed_compund_csv.to_csv(output)'

The sets of reactions that are described and undescribed in ModelSEED for each NIST enzyme are exported.

In [88]:
from ftfy import fix_encoding
from datetime import date
import json

nist = json.load(open('2021-05-06_NIST consolidated.json'))
described_compounds = json.load(open('2021-06-27_APF_described compounds in ModelSEED_01.json'))

described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.extend(described_compounds[compound]['alternative names'])

#print('Described compounds: {}\n'.format(len(described_compounds_list)), described_compounds_list)
    
reactions = {}
repeated_reactions = set()
enzymes = set()
for enzyme, information in nist.items():
    enzymes.add(enzyme)
    undescribed_reactions = []
    described_reactions = []
    for reaction in information['reaction']:
        reaction = fix_encoding(reaction)
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            compounds = reactants_list + products_list
            for element in compounds:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element not in compounds_list:
                    compounds_list.append(element)

        #print(compounds_list)
        described = all(compound in described_compounds_list for compound in compounds_list)
        if described and reaction not in (undescribed_reactions and described_reactions):
            described_reactions.append(reaction)
        elif not described and reaction not in (undescribed_reactions and described_reactions):
            undescribed_reactions.append(reaction)
        else:
            repeated_reactions.add(reaction)
                
        '''else:
            undescribed_reactions[element] = {'Enzyme': enzyme,
                                              'Reaction': reaction,
                                              'synonym': ''}
        
            continue'''
        
    reactions[enzyme] = {'completely described reactions': described_reactions,
                         'undescribed reactions': undescribed_reactions}

completely_described = 0
undescribed = 0
for enzyme, data in reactions.items():
    if len(data['completely described reactions']) != 0 and len(data['undescribed reactions']) == 0:
        completely_described += 1
    else:
        undescribed += 1
        
print('completely described reactions: ', completely_described)
print('undescribed reactions: ', undescribed)
print('repeated reactions: ', len(repeated_reactions))
print('total enzymes: ', len(enzymes))
    
# export the described and undescribed reactions
with open('{}_APF_NIST reactions in ModelSEED_01.json'.format(date.today()), 'w') as output:
    json.dump(reactions, output, indent = 4)

completely described reactions:  147
undescribed reactions:  278
repeated reactions:  41
total enzymes:  425


The data for each completely described reaction is acquired and exported.  

In [89]:
from datetime import date
import json
import re

def isfloat(entry):
    try:
        value = float(entry)
        return True
    except ValueError:
        value = ''
        return False


reactions = json.load(open('2021-06-27_APF_NIST reactions in ModelSEED_01.json'))
nist = json.load(open('2021-05-06_NIST consolidated.json'))

described_enzymes_list = []
for enzyme, data in reactions.items():
    described_enzymes_list.append(enzyme.lower())
    
described_enzyme_data = {}
for enzyme, information in nist.items():
    if enzyme.lower() in described_enzymes_list:
        
        # determine the set of unique reactions for each enzyme
        unique_reactions = []
        for reaction in information['reaction']:
            if reaction not in unique_reactions:
                unique_reactions.append(reaction)
        
        # calculate the average temperature 
        temperatures = []
        for temperature in information['experimental temperatures']:
            temperature = re.sub('(~)', '', temperature)
            if isfloat(temperature):
                temperatures.append(float(temperature))
            else:
                print(temperature)
        
        if len(temperatures) > 0:
            average_temperature = sum(temperatures) / len(temperatures)
        else:
            average_temperautre = 'nan'
            
        # calculate the average ph 
        phs = []
        for ph in information['experimental phs']:
            ph = re.sub('(~|nan)', '', ph)
            if isfloat(ph):
                phs.append(float(ph))
            
        if len(phs) > 0:
            average_ph = sum(phs) / len(phs)
        else:
            average_ph = 'nan'
        
    described_enzyme_data[enzyme] = {'unique reactions': unique_reactions,
                                    'average keq': information['Keq']['keq average'],
                                    'average enthalpy': information['Enthalpy']['enthalpy average'],
                                    'average temperature kelvin': average_temperature,
                                    'average pH': average_ph}

with open('{}_APF_ModelSEED described enzymes_01.json'.format(date.today()), 'w') as output:
    json.dump(described_enzyme_data, output, indent = 4)

298.l5


The NIST reactions are partitioned into a TSV file, following the reaction example https://github.com/ModelSEED/ModelSEEDDatabase/blob/chemistry_staging/Biochemistry/Curation/janakagithub/CentralMetabolism_InorganicMetabolism.tsv. The ModelSEED ID substitution occurs in the subsequent cells.

In [90]:
from datetime import date
import json

nist_data = json.load(open('2021-06-27_APF_ModelSEED described enzymes_01.json'))
     
with open('{}_APF_ModelSEED formatted reactions.tsv'.format(date.today()), 'w') as output:    
    output.write('\t'.join(['ID', 'Names', 'Equation defintion', '\n']))
    id_number = 0
    for enzyme, information in nist_data.items():
        for reaction in information['unique reactions']:            
            id = 'TECRDB{}'.format(id_number)
            output.write('\t'.join([id, enzyme, reaction]) + '\n')
            id_number += 1
    
    output.close()

The reactions were completely prepared for passage through the ModelSEED code. The compounds were all substituted with the ModelSEED IDs and the reaction string format was aligned with the ModelSEED conventions. The TSV file will be exported and then executed through the ModelSEED Add_new_curated_reactions.py file. 

In [109]:
from ftfy import fix_encoding
from datetime import date
import json
import re

nist_reactions = json.load(open('2021-06-27_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-06-27_APF_described compounds in ModelSEED_01.json'))
described_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))

described_compounds_list = []
modelseed_simplified = {}
for compound in nist_compounds:
    described_compounds_list.extend([compound.lower()] + list(nist_compounds[compound]['alternative names']))
    
total_complete_reactions = []
exchanged_reactions = set()
to_modelseed_reactions = {}

reaction_id_pairs = {}
id_number = 0
for enzyme, information in nist_reactions.items():
    #if enzyme in described_reactions.keys():
    #print('\n\n{}\n'.format(enzyme))
    #for modelseed_id in information['ModelSEED ID']:
    complete_reactions = []
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
        #print('reaction: {}'.format(reaction))

        # parse the reactants and products for the specified reaction string
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            stoich_coefficient = re.compile('(\d\s|\d\/\d\s)')
            for element in reactants_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                
                if element in described_compounds_list:
                    for compound, aliases in nist_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            reactants.append(coefficient + aliases['ModelSEED ID'])
                        
                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub(stoich_coefficient, '', element)
                if element in described_compounds_list:
                    for compound, aliases in nist_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            products.append(coefficient + aliases['ModelSEED ID'])
                else:
                    products.append(element)

            compounds = reactants + products
        
        else:
            print('incorrect reaction: ', reaction)
            
        # assemble the reaction string
        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
        elif len(reactants) == 1:
            reactants_string = reactants[0]
        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        if len(products) > 1:
            products_string = ' + '.join(products)
        elif len(products) == 1:
            products_string = products[0]
        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        reaction_string = reactants_string + ' <=> ' + products_string
            
        # format the reaction string according to ModelSEED conventions of the stoichiometric coefficients and the compartment signifier                        
        reaction_string = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'({}) '.format(coefficient), reaction_string)
        reaction_string = re.sub('(?<=\d)(\s)(?=\))', '', reaction_string)
            
        if re.search('(?<=>|\+)(\s)(?=\w)', reaction_string):
            reaction_string = re.sub('(?<=>|\+)(\s)(?=\w)', ' (1) ', reaction_string)

        if not re.search('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string
            
        reaction_string = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction_string)
        reaction_string += '[0]'
            
        #print('''reaction_string: {}
        #products_string: {}'''.format(type(reaction_string), type(products_string)))
        #print('exchanged reaction: {}'.format(final_reaction))
        
        
        exchanged_reactions.add(reaction_string)
        if all('cpd' in compound for compound in compounds):
            print(reaction_string)
            complete_reactions.append(reaction_string)

        
        '''for compound, ids in nist_compounds.items():
            if re.search(compound, reaction, re.IGNORECASE):
                if not re.search('\w {}'.format(compound), reaction, re.IGNORECASE):
                reaction = re.sub(compound, ids['ModelSEED ID'], reaction)'''

    reaction_ids = []
    for reaction in complete_reactions:
        if reaction not in reaction_id_pairs.values():
            reaction_id = 'TECRDB{}'.format(id_number)
            reaction_id_pairs[reaction_id] = reaction
            id_number += 1
        else:
            index = list(reaction_id_pairs.values()).index(reaction)
            reaction_id = list(reaction_id_pairs.keys())[index]
            print('--> {} is a redundant reaction.'.format(reaction))
            
        reaction_ids.append(reaction_id)

    to_modelseed_reactions[enzyme] = {'reactions':complete_reactions, 'ids':reaction_ids}
    total_complete_reactions += complete_reactions
    
#print(exchanged_reactions)
total_complete_reactions_set = set(total_complete_reactions)
    
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Total complete reactions: {}'.format(len(total_complete_reactions)))
print('Total complete reactions set: {}'.format(len(total_complete_reactions_set)))
print('Previously complete reactions: {}'.format(len(set(list(described_reactions.keys())))))
#print(complete_reactions)


'''with open('{}_APF_NIST to ModelSEED, in form_01.json'.format(date.today()), 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)'''

with open('{}_APF_NIST to ModelSEED enzymes with reactions-ids.json'.format(date.today()), 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)

with open('{}_APF_NIST to ModelSEED reaction-id dictionary.json'.format(date.today()), 'w') as output:
    json.dump(reaction_id_pairs, output, indent = 4)


# export the ModelSEED formatted reactions into a TSV file 
tsv_lines = []
'''for enzyme, information in to_modelseed_reactions.items():
    for reaction in information:'''
for reaction_id in reaction_id_pairs:
    tsv_lines.append('\t'.join([reaction_id, reaction_id_pairs[reaction_id]]))
    
print('Total reactions in TSV: {}'.format(len(tsv_lines)))  
        
tsv_file = open('{}_APF_NIST to ModelSEED, in form, with ids_01.tsv'.format(date.today()), 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()
    

'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

(1) cpd00041[0] <=> (1) cpd00106[0] + (1) cpd00013[0]
(1) cpd00482[0] <=> (1) cpd00061[0] + (1) cpd00001[0]
(1) cpd00482[0] <=> (1) cpd00169[0]
(1) cpd03535[0] + (1) cpd00001[0] <=> (1) cpd00051[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd01550[0] + (1) cpd00001[0] <=> (1) cpd00250[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd00061[0] + (1) cpd00001[0] <=> (1) cpd00020[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd00818[0] + (1) cpd00001[0] <=> (1) cpd00709[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd02135[0] + (1) cpd00001[0] <=> (1) cpd00646[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd02023[0] + (1) cpd00001[0] <=> (1) cpd00451[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0]
(1) cpd00012[0] + (1) cpd00012[0] + (1) cpd00001[0] <=> (2) cpd00009[0] + (2) cpd00009[0] + (2) cpd00009[0]
(1) cpd00457[0] + (1) cpd00001[0] <=> (1) cpd00098[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd0

(1) cpd27436[0] + (1) cpd00003[0] <=> (1) cpd19035[0] + (1) cpd00004[0]
(1) cpd00139[0] + (1) cpd00003[0] <=> (1) cpd00040[0] + (1) cpd00004[0]
--> (1) cpd00139[0] + (1) cpd00003[0] <=> (1) cpd00040[0] + (1) cpd00004[0] is a redundant reaction.
(1) cpd00033[0] + (1) cpd00055[0] <=> (1) cpd00054[0]
(1) cpd00033[0] + (1) cpd00071[0] <=> (1) cpd00161[0]
(1) cpd00125[0] + (1) cpd00033[0] + (1) cpd00001[0] <=> (1) cpd00087[0] + (1) cpd00087[0] + (1) cpd00054[0]
(1) cpd00808[0] <=> (1) cpd00198[0]
(1) cpd01627[0] <=> (1) cpd00642[0]
(1) cpd00002[0] + (1) cpd00655[0] <=> (1) cpd00008[0] + (1) cpd22528[0]
(1) cpd00304[0] + (1) cpd00304[0] <=> (1) cpd01430[0]
(1) cpd00171[0] <=> (1) cpd00198[0]
(1) cpd00368[0] <=> (1) cpd00259[0]
(1) cpd00051[0] + (1) cpd00033[0] <=> (1) cpd00064[0] + (1) cpd00451[0]
(1) cpd00353[0] <=> (1) cpd00036[0] + (1) cpd00040[0]
(1) cpd00137[0] <=> (1) cpd29968[0] + (1) cpd00032[0] + (1) cpd00032[0]
(1) cpd00540[0] + (1) cpd00135[0] <=> (1) cpd00756[0] + (1) cpd00060[0]

(1) cpd01876[0] <=> (1) cpd01545[0]
(1) cpd00002[0] + (1) cpd00126[0] <=> (1) cpd00008[0] + (1) cpd00031[0]
(1) cpd00002[0] + (1) cpd00296[0] <=> (1) cpd00008[0] + (1) cpd00295[0]
(1) cpd00020[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0] <=> (1) cpd00196[0] + (1) cpd00047[0]
(1) cpd00012[0] + (1) cpd00012[0] + (1) cpd00054[0] <=> (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00738[0] + (1) cpd00738[0]
(1) cpd02129[0] + (1) cpd00006[0] <=> (1) cpd01964[0] + (1) cpd00005[0]
(1) cpd00025[0] <=> (0.5) cpd00007[0] + (1) cpd00001[0]
(1) cpd09394[0] + (7) cpd00001[0] <=> (7) cpd19001[0]
(1) cpd24451[0] + (6) cpd00001[0] <=> (6) cpd19001[0]
(1) cpd00032[0] + (1) cpd00032[0] + (1) cpd00022[0] + (1) cpd00001[0] <=> (1) cpd00137[0] + (1) cpd22528[0]
(1) cpd00125[0] + (1) cpd00299[0] <=> (1) cpd00330[0] + (1) cpd00330[0] + (1) cpd00298[0]
(1) cpd03583[0] <=> (1) cpd00171[0] + (1) cpd00055[0]
(1) cpd03583[0] <=> (1) cpd19035[0]
(1) cpd00022[0] + (1) cpd00047[0] <=> (1) cp

"with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:\n    json.dump( , output, indent = 4)"

# Brainstorming

Conceptualizing a scrapping code for KEGG, which would unambiguously identify the NIST compounds.

In [ ]:
import BeautifulSoup
import selenium
import json

compounds = json.load(open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json'))

num = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

general_delay = 2  #seconds



def click_element_type(n_type):
    global driver
    element = driver.find_element_by_xpath(n_type)
    element.click()
    time.sleep(general_delay)
    
def select_dropdown_id(n_id, n_choice):
    global driver
    element = Select(driver.find_element_by_id(n_id))
    element.select_by_visible_text(n_choice)
    time.sleep(general_delay)

def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
        

def scrape(compound):
    global driver
    global enzymes
    
    driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
    driver.get("https://www.genome.jp/kegg/")
    
    time.sleep(general_delay)
    
    # search field
    text_area = driver.find_element_by_name("text")
    text_area.send_keys(compound)
    time.sleep(general_delay)
    click_element_id("//input[@value='Search']")
    time.sleep(general_delay)
    
    # display all compound button
    click_element_type("/html/body/form/a[@href='javascript:MoreSearch(\"compound\")']")
    
    # acquire the list of results
    '''possible_compounds = []
    for div in click_element_type("/html/body/form/div[@style='width_600px']"):'''
    url = driver.current_url
    
    
        

    compounds[compound] = "no"
    driver.close()
    return

    
    time.sleep(general_delay)
    
    select_dropdown_id("max", "100")
    element = Select(driver.find_element_by_id("max"))
    element.select_by_visible_text("100")
    
    time.sleep(general_delay)


    else:
        compounds[compound] = "no"
        driver.close()
        return
    
    enzymes[enzyme_name] = "yes"
    
    driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")
    
    time.sleep(15)
    
    click_element_id("excelExport")
    
    time.sleep(5)
    
    driver.close()
    


for compound in compounds:
    if compounds[compound] != "yes" and compounds[compound] != "no":
        flag = True
        while flag:
            try:
                scrape(compound)
                flag = False
            except:
                time.sleep(10)
            
        with open('compounds_to_collect_out.json', 'w') as outfile:
            json.dump(compounds, outfile, indent = 3)   
            outfile.close()



In [3]:
print('\xb0K')

°K


In [29]:
import re

compound = 'hi, yes, no'
re.sub('(^|, )(\S+)\w', '\u\1', ', {}, '.format(compound))
print(compound)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (<ipython-input-29-f8fcd23c4c81>, line 4)

In [99]:
test = ['1(-)wer3', 'asdf2-33d', 'asdf2-33d', 's-methylthioglycolate', '(-)-ureidoglycolate']

test_set = set(test)
print(test_set)

{'asdf2-33d', '1(-)wer3', '(-)-ureidoglycolate', 's-methylthioglycolate'}
